In [ ]:
!pip install --upgrade google-cloud-storage

### Machine possibilities research:

In [ ]:
!nvidia-smi

Thu Mar  6 14:45:28 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   37C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
import torch
batch_size = 1
while True:
    try:
        x = torch.randn(batch_size, 1, 16000).to("cuda")
        batch_size *= 2
    except RuntimeError:
        print(f"Max batch size before OOM: {batch_size // 2}")
        break

KeyboardInterrupt: 

In [ ]:
import multiprocessing
print(multiprocessing.cpu_count())

8


### .

In [ ]:
!pip install datasets

In [ ]:
from google.colab import auth

In [ ]:
auth.authenticate_user()

In [ ]:
!gcloud auth login --quiet

Go to the following link in your browser, and complete the sign-in prompts:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=32555940559.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fauthcode.html&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=wx5xnvjJFsU0sQ7b0zpfFvKmbodyqH&prompt=consent&token_usage=remote&access_type=offline&code_challenge=gZ9YPSITutPC5xn2eDyfxhomD35-tPQuug2PA_IqQt8&code_challenge_method=S256

Once finished, enter the verification code provided in your browser: 4/0AQSTgQHsvanlGYA_Csk6hMmJ_mygD0AdcZaRWCggXC2uJ6lUlUurNHw8foR_9AdMVRsgKw

You are now logged in as [khrystyna.dol.3@gmail.com].
Your current proj

In [ ]:
!gcloud config set project ser-diploma

Updated property [core/project].


In [ ]:
from google.cloud import storage

client = storage.Client()
bucket_name = "mc-eiu-data"

bucket = client.bucket(bucket_name)
blobs = bucket.list_blobs()

for blob in blobs:
    print(blob.name)
    break

EnglishDialogues.csv


### Dataset processing:

In [ ]:
import os
import pandas as pd
from google.cloud import storage

BUCKET_NAME = "mc-eiu-data"
CSV_FILE_PATH = f"EnglishDialogues.csv"
AUDIO_FOLDER_PATH = "MC-EIU-audio/"

storage_client = storage.Client()
bucket = storage_client.bucket(BUCKET_NAME)

blob = bucket.blob(CSV_FILE_PATH)
csv_content = blob.download_as_text()
df = pd.read_csv(pd.io.common.StringIO(csv_content))

def get_audio_filename(dia, utt):
    return f"dia_{dia}_utt_{utt}.mp3"

def file_exists_in_bucket(filename):
    return storage.Blob(bucket=bucket, name=f"{AUDIO_FOLDER_PATH}{filename}").exists(storage_client)

dataset = []
idx = 0
for _, row in df.iterrows():
    audio_filename = get_audio_filename(row["Dia_No"], row["Utt_No"])

    if file_exists_in_bucket(audio_filename):
        dataset.append({
            "audio_file": f"gs://{BUCKET_NAME}/{AUDIO_FOLDER_PATH}{audio_filename}",
            "text": row["Script"],
            "emotion": row["emotion"],
            "intent": row["intent"],
            "speaker": row["speaker"],
            "dialogue_id": row["Dia_No"],
            "utterance_id": row["Utt_No"]
        })
    if idx % 1000 == 0:
      print("Files processed", idx)
    idx += 1

dataset_df = pd.DataFrame(dataset)

print(f"Loaded dataset with {len(dataset_df)} matched samples.")
dataset_df.head()

Files processed 0
Files processed 500
Files processed 1000
Files processed 1500
Files processed 2000
Files processed 2500
Files processed 3000
Files processed 3500
Files processed 4000
Files processed 4500
Files processed 5000
Files processed 5500
Files processed 6000
Files processed 6500
Files processed 7000
Files processed 7500
Files processed 8000
Files processed 8500
Files processed 9000
Files processed 9500
Files processed 10000
Files processed 10500
Files processed 11000
Files processed 11500
Files processed 12000
Files processed 12500
Files processed 13000
Files processed 13500
Files processed 14000
Files processed 14500
Files processed 15000
Files processed 15500
Files processed 16000
Files processed 16500
Files processed 17000
Files processed 17500
Files processed 18000
Files processed 18500
Files processed 19000
Files processed 19500
Files processed 20000
Files processed 20500
Files processed 21000
Files processed 21500
Files processed 22000
Files processed 22500
Files proces

,audio_file,text,emotion,intent,speaker,dialogue_id,utterance_id
0,gs://mc-eiu-data/MC-EIU-audio/dia_1_utt_0.mp3,"Ross: Hi... There she is. Hi Emma. Oh my God,...",happy,questioning,0,1,0
1,gs://mc-eiu-data/MC-EIU-audio/dia_1_utt_1.mp3,"Rachel: That was one time, Ross, and they were...",neutral,neutral,1,1,1
2,gs://mc-eiu-data/MC-EIU-audio/dia_1_utt_2.mp3,"Ross: Ooh hey, Emma, daddy has some presents f...",neutral,wishing,0,1,2
3,gs://mc-eiu-data/MC-EIU-audio/dia_1_utt_3.mp3,"Rachel: Aaah... Ross, actually there's somethi...",anger,neutral,1,1,3
4,gs://mc-eiu-data/MC-EIU-audio/dia_1_utt_4.mp3,"Ross: Okay, shoot! \n",neutral,agreeing,0,1,4


In [ ]:
dataset_df["emotion"].value_counts()

,count
emotion,
neutral,21428
happy,12622
anger,4476
sad,2649
surprise,1732
fear,1256
disgust,844


In [ ]:
dataset_df["intent"].value_counts()

,count
intent,
neutral,17767
questioning,10131
suggesting,4748
consoling,4009
agreeing,2997
wishing,2714
encouraging,1792
acknowledging,584
sympathizing,265


In [ ]:
emotion_to_label = {
    "neutral": 0, "happy": 1, "anger": 2, "sad": 3,
    "surprise": 4, "fear": 5, "disgust": 6
}

dataset_df["emotion_label"] = dataset_df["emotion"].map(emotion_to_label).fillna(-1).astype(int)
dataset_df.head()

,audio_file,text,emotion,intent,speaker,dialogue_id,utterance_id,emotion_label
0,gs://mc-eiu-data/MC-EIU-audio/dia_1_utt_0.mp3,"Ross: Hi... There she is. Hi Emma. Oh my God,...",happy,questioning,0,1,0,1
1,gs://mc-eiu-data/MC-EIU-audio/dia_1_utt_1.mp3,"Rachel: That was one time, Ross, and they were...",neutral,neutral,1,1,1,0
2,gs://mc-eiu-data/MC-EIU-audio/dia_1_utt_2.mp3,"Ross: Ooh hey, Emma, daddy has some presents f...",neutral,wishing,0,1,2,0
3,gs://mc-eiu-data/MC-EIU-audio/dia_1_utt_3.mp3,"Rachel: Aaah... Ross, actually there's somethi...",anger,neutral,1,1,3,2
4,gs://mc-eiu-data/MC-EIU-audio/dia_1_utt_4.mp3,"Ross: Okay, shoot! \n",neutral,agreeing,0,1,4,0


In [ ]:
dataset_df["emotion_label"].value_counts()

,count
emotion_label,
0,21428
1,12622
2,4476
3,2649
4,1732
5,1256
6,844


In [ ]:
local_csv_path = "/content/processed_dataset.csv"
dataset_df.to_csv(local_csv_path, index=False)
print(f"Saved locally at: {local_csv_path}")

OUTPUT_CSV_PATH = "MC-EIU-processed.csv"

output_blob = bucket.blob(OUTPUT_CSV_PATH)
output_blob.upload_from_filename(local_csv_path)
print(f"Uploaded to GCS: gs://{BUCKET_NAME}/{OUTPUT_CSV_PATH}")

Saved locally at: /content/processed_dataset.csv
Uploaded to GCS: gs://mc-eiu-data/MC-EIU-processed.csv


### Loading dataset from gcp bucket:

In [ ]:
import pandas as pd
import gcsfs

fs = gcsfs.GCSFileSystem(project='ser-diploma')
gcs_path = 'gs://mc-eiu-data/MC-EIU-processed.csv'

with fs.open(gcs_path, 'r') as f:
    df = pd.read_csv(f)

df.head()

,audio_file,text,emotion,intent,speaker,dialogue_id,utterance_id,emotion_label
0,gs://mc-eiu-data/MC-EIU-audio/dia_1_utt_0.mp3,"Ross: Hi... There she is. Hi Emma. Oh my God,...",happy,questioning,0,1,0,1
1,gs://mc-eiu-data/MC-EIU-audio/dia_1_utt_1.mp3,"Rachel: That was one time, Ross, and they were...",neutral,neutral,1,1,1,0
2,gs://mc-eiu-data/MC-EIU-audio/dia_1_utt_2.mp3,"Ross: Ooh hey, Emma, daddy has some presents f...",neutral,wishing,0,1,2,0
3,gs://mc-eiu-data/MC-EIU-audio/dia_1_utt_3.mp3,"Rachel: Aaah... Ross, actually there's somethi...",anger,neutral,1,1,3,2
4,gs://mc-eiu-data/MC-EIU-audio/dia_1_utt_4.mp3,"Ross: Okay, shoot! \n",neutral,agreeing,0,1,4,0


In [ ]:
dataset_df = df[['audio_file', 'emotion_label']].copy()
dataset_df.head()

,audio_file,emotion_label
0,gs://mc-eiu-data/MC-EIU-audio/dia_1_utt_0.mp3,1
1,gs://mc-eiu-data/MC-EIU-audio/dia_1_utt_1.mp3,0
2,gs://mc-eiu-data/MC-EIU-audio/dia_1_utt_2.mp3,0
3,gs://mc-eiu-data/MC-EIU-audio/dia_1_utt_3.mp3,2
4,gs://mc-eiu-data/MC-EIU-audio/dia_1_utt_4.mp3,0


### Loading data and model training

In [ ]:
# train_ratio = 0.8
# dataset_shuffled = dataset_df.sample(frac=1, random_state=42).reset_index(drop=True)
# split_idx = int(len(dataset_shuffled) * train_ratio)

# train_dataset = dataset_shuffled[:split_idx]
# test_dataset = dataset_shuffled[split_idx:]

# print(f"Train set: {len(train_dataset)} samples, Test set: {len(test_dataset)} samples")

train_ratio = 0.6
val_ratio = 0.2

dataset_shuffled = dataset_df.sample(frac=1, random_state=42).reset_index(drop=True)

train_idx = int(len(dataset_shuffled) * train_ratio)
val_idx = int(len(dataset_shuffled) * (train_ratio + val_ratio))

train_dataset = dataset_shuffled[:train_idx]
val_dataset = dataset_shuffled[train_idx:val_idx]
test_dataset = dataset_shuffled[val_idx:]

print(f"Train set: {len(train_dataset)} samples")
print(f"Validation set: {len(val_dataset)} samples")
print(f"Test set: {len(test_dataset)} samples")

Train set: 27004 samples
Validation set: 9001 samples
Test set: 9002 samples


In [ ]:
train_dataset["emotion_label"].value_counts(normalize=True) * 100

,proportion
emotion_label,
0,47.370760
1,28.095838
2,10.187380
3,5.862095
4,3.803140
5,2.881055
6,1.799733


In [ ]:
test_dataset["emotion_label"].value_counts(normalize=True) * 100

,proportion
emotion_label,
0,48.033770
1,27.849367
2,9.753388
3,5.854255
4,3.854699
5,2.688291
6,1.966230


In [ ]:
import io
import torch
import librosa
import torchaudio
import numpy as np
import multiprocessing
import soundfile as sf
import tensorflow as tf
from datasets import Dataset
from transformers import Wav2Vec2FeatureExtractor

def map_to_array(example):
    gcs_path = example["audio_file"]

    with tf.io.gfile.GFile(gcs_path, "rb") as f:
        audio_bytes = f.read()

    audio, samplerate = sf.read(io.BytesIO(audio_bytes), always_2d=True)

    if audio.shape[1] > 1:
        audio = audio.mean(axis=1)

    if samplerate != 16000:
        audio = librosa.resample(audio, orig_sr=samplerate, target_sr=16000)

    example["speech"] = audio
    return example

num_proc = min(multiprocessing.cpu_count() // 2, 4)
print("Number of process used:", num_proc)

# train_data = Dataset.from_pandas(train_dataset).map(map_to_array)
# val_data = Dataset.from_pandas(val_dataset).map(map_to_array)
# test_data = Dataset.from_pandas(test_dataset).map(map_to_array)

train_data = Dataset.from_pandas(train_dataset).map(map_to_array, keep_in_memory=False)
val_data = Dataset.from_pandas(val_dataset).map(map_to_array, keep_in_memory=False)
test_data = Dataset.from_pandas(test_dataset).map(map_to_array, keep_in_memory=False)

# train_data = Dataset.from_pandas(train_dataset).map(map_to_array, num_proc=num_proc)
# val_data = Dataset.from_pandas(val_dataset).map(map_to_array, num_proc=num_proc)
# test_data = Dataset.from_pandas(test_dataset).map(map_to_array, num_proc=num_proc)

# train_data = Dataset.from_pandas(train_dataset).map(map_to_array, num_proc=num_proc, keep_in_memory=False)
# val_data = Dataset.from_pandas(val_dataset).map(map_to_array, num_proc=num_proc, keep_in_memory=False)
# test_data = Dataset.from_pandas(test_dataset).map(map_to_array, num_proc=num_proc, keep_in_memory=False)

Number of process used: 4


Map:   0%|          | 0/27004 [00:00<?, ? examples/s]

Map:   0%|          | 0/9001 [00:00<?, ? examples/s]

Map:   0%|          | 0/9002 [00:00<?, ? examples/s]

In [ ]:
import torch

def encode_batch(batch, feature_extractor, device="cuda"):
    batch_size = 32
    encodings = []

    for i in range(0, len(batch), batch_size):
        batch_subset = batch[i : i + batch_size]
        encoding = feature_extractor(
            batch_subset, sampling_rate=16000, padding=True, return_tensors="pt"
        )
        encoding = {k: v.to(device) for k, v in encoding.items()}
        encodings.append(encoding)

    return encodings

num_labels = 7
feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained("superb/hubert-large-superb-er")  # , num_labels=num_labels)

train_encodings = encode_batch(list(train_data["speech"]), feature_extractor)
val_encodings = encode_batch(list(val_data["speech"]), feature_extractor)
test_encodings = encode_batch(list(test_data["speech"]), feature_extractor)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
# num_labels=7
# feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained("superb/hubert-large-superb-er")  # , num_labels=num_labels)
# feature_extractor = feature_extractor.to("cuda")

# train_encodings = feature_extractor(list(train_data["speech"]), sampling_rate=16000, padding=True, return_tensors="pt")
# test_encodings = feature_extractor(list(test_data["speech"]), sampling_rate=16000, padding=True, return_tensors="pt")

num_labels = 7
feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained("superb/hubert-large-superb-er")  # , num_labels=num_labels)

train_encodings = feature_extractor(list(train_data["speech"]), sampling_rate=16000, padding=True, return_tensors="pt").to("cuda")
# train_encodings = {k: v.to("cuda") for k, v in train_encodings.items()}

val_encodings = feature_extractor(list(val_data["speech"]), sampling_rate=16000, padding=True, return_tensors="pt").to("cuda")

test_encodings = feature_extractor(list(test_data["speech"]), sampling_rate=16000, padding=True, return_tensors="pt").to("cuda")
# test_encodings = {k: v.to("cuda") for k, v in test_encodings.items()}

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/212 [00:00<?, ?B/s]

In [ ]:
import torch

class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.pattern = {1: 0, 2: 1, 3: 2, 4: 3, 5: 4, 6: 5, 7: 6}
        # self.labels = [pattern[x] for x in labels]
        self.labels = [self.pattern.get(x) for x in labels]

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['emotion_label'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)


train_dataset = CustomDataset(train_encodings, list(train_data["emotion_label"]))
test_dataset = CustomDataset(test_encodings, list(test_data["emotion_label"]))

In [ ]:
from torch.optim import AdamW
from transformers import HubertForSequenceClassification

num_labels=7
model = HubertForSequenceClassification.from_pretrained("superb/hubert-large-superb-er", num_labels=num_labels, ignore_mismatched_sizes=True)
# model = HubertForSequenceClassification.from_pretrained("facebook/hubert-large-ls960-ft", num_labels=8)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

optim = AdamW(model.parameters(), lr=1e-5)

In [ ]:
def predict(outputs):
    probabilities = torch.softmax(outputs["logits"], dim=1)
    predictions = torch.argmax(probabilities, dim=1)
    return predictions

In [ ]:
import torch
import time
from torch.utils.data import DataLoader
from tqdm import tqdm

epoch = 5
patience = 2
best_val_loss = float("inf")
early_stopping_counter = 0
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

train_loss = []
train_accuracies = []
val_losses = []

for epoch_i in range(epoch):
    print(f'Epoch {epoch_i + 1}/{epoch}')
    time.sleep(0.3)

    model.train()
    train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True)

    correct = 0
    count = 0
    epoch_loss = []

    pbar = tqdm(train_loader)
    for batch in pbar:
        optim.zero_grad()

        input_ids = batch['input_values'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['Emotion_label'].to(device)

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optim.step()

        predictions = predict(outputs)

        correct += predictions.eq(labels).sum().item()
        count += len(labels)
        accuracy = correct / count

        pbar.set_postfix({
            'Loss': f'{loss.item():.3f}',
            'Accuracy': f'{accuracy:.3f}'
        })

        epoch_loss.append(loss.item())
    pbar.close()

    train_loss.append(sum(epoch_loss) / len(epoch_loss))
    train_accuracies.append(accuracy)

    model.eval()
    val_loss = []
    val_loader = DataLoader(val_dataset, batch_size=2, shuffle=False)

    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch['input_values'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['Emotion_label'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            val_loss.append(loss.item())

    avg_val_loss = sum(val_loss) / len(val_loss)
    val_losses.append(avg_val_loss)

    print(f'Validation Loss: {avg_val_loss:.4f}')

    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        torch.save(model.state_dict(), "mc-eiu-hubert-model.pth")
        print("Best model saved.")
        early_stopping_counter = 0
    else:
        early_stopping_counter += 1

    # Early Stopping
    if early_stopping_counter >= patience:
        print("Early stopping triggered. Training stopped.")
        break

print("Training complete.")

In [ ]:
# from torch.utils.data import DataLoader

# epoch = 5
# model.train()

# train_loss = list()
# train_accuracies = list()
# for epoch_i in range(epoch):
#     print('Epoch %s/%s' % (epoch_i + 1, epoch))
#     time.sleep(0.3)

#     train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True)

#     correct = 0
#     count = 0
#     epoch_loss = list()

#     pbar = tqdm(train_loader)
#     for batch in pbar:
#         optim.zero_grad()
#         input_ids = batch['input_values'].to(device)
#         attention_mask = batch['attention_mask'].to(device)
#         labels = batch['Emotion_label'].to(device)
#         outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
#         loss = outputs['loss']
#         loss.backward()
#         optim.step()

#         predictions = predict(outputs)

#         correct += predictions.eq(labels).sum().item()
#         count += len(labels)
#         accuracy = correct * 1.0 / count

#         pbar.set_postfix({
#             'Loss': '{:.3f}'.format(loss.item()),
#             'Accuracy': '{:.3f}'.format(accuracy)
#         })

#         epoch_loss.append(loss.item())
#     pbar.close()

#     train_loss += epoch_loss
#     train_accuracies.append(accuracy)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Plot Iteration vs Training Loss
plt.plot(train_loss, label="Training Loss")
plt.xlabel("Iteration")
plt.ylabel("Loss")
plt.title("Iteration vs Training Loss")
plt.legend()
plt.show()

# Plot Epoch vs Training Accuracy
acc_X_1 = np.arange(len(train_accuracies))+1
plt.plot(acc_X_1, train_accuracies,"-", label="Training Accuracy")
plt.xticks(acc_X_1)
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.title("Epoch vs Training Accuracy")
plt.legend()
plt.show()

print(acc_X_1)
train_accuracies1=train_accuracies
print(train_accuracies1)

In [ ]:
from imblearn.over_sampling import SMOTE
from collections import Counter

X = dataset_df.drop(columns=["emotion_label"])
y = dataset_df["emotion_label"]

smote = SMOTE(sampling_strategy='auto', random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

df_balanced = pd.DataFrame(X_resampled)
df_balanced["emotion_label"] = y_resampled

print("Class distribution after SMOTE:", Counter(y_resampled))

ValueError: could not convert string to float: 'gs://mc-eiu-data/MC-EIU-audio/dia_1_utt_0.mp3'

In [ ]:
local_csv_path = "/content/resampled_dataset.csv"
df.to_csv(local_csv_path, index=False)
print(f"Saved locally at: {local_csv_path}")

OUTPUT_CSV_PATH = "MC-EIU-resampled.csv"

output_blob = bucket.blob(OUTPUT_CSV_PATH)
output_blob.upload_from_filename(local_csv_path)
print(f"Uploaded to GCS: gs://{BUCKET_NAME}/{OUTPUT_CSV_PATH}")